# Example: 4 players 2 actions, degenerate

In [1]:
import numpy as np
from sage.all import *
from gtnash.game.hypergraphicalgame  import *
from gtnash.util.irda import *
from gtnash.util.polynomial_complementary_problem import *
from gtnash.solver.wilsonalgorithm import *


def present_node(current_node,y_values):
    print(f"Level: {current_node.level}")
    var_pl=[]
    var_notpl=[]
    for (n,i) in current_node.pcp.couple_to_x.keys():
        if n>current_node.level:
            var_notpl+=[current_node.pcp.couple_to_x[(n,i)]]
        else:
            var_pl+=[current_node.pcp.couple_to_x[(n,i)]]
    print(f"Variables of players at the current level: {var_pl}")
    print(f"Variables of players above the current level:{var_notpl}")
    print("Polynomials:")
    for pol in current_node.pcp.couple_to_poly.keys(): 
        print(" "+str(pol)+": "+str(current_node.pcp.couple_to_poly[pol].subs(y_values)))

    print(f"Set Z: {current_node.set_z}")
    print(f"Set W: {current_node.set_w}")
    print(f"Coordinate of the node: {current_node.coordinates}")

    print("Equations:")
    for e in range(len(current_node.sys_S.equations)):
        if not current_node.sys_S.equations[e].subs(y_values)==0:
            print(f" {current_node.sys_S.equations[e].subs(y_values)} = 0")
    print("Inequations:")
    for (n,i) in current_node.pcp.couple_to_x.keys():
        if not (n,i) in current_node.set_z:
            print(f" {current_node.pcp.couple_to_x[n,i]} >= 0")
    for e in range(len(current_node.sys_S.inequations)):
        print(f" {current_node.sys_S.inequations[e].subs(y_values)} >=0 ")


## Definition of the game:

Before creating the PCP of a game, we first compute its disutility:

In [ ]:
utilities=[[50, 59, 27, 4, 72, 69, 18, 35, 6, 89, 78, 19, 36, 76, 64, 20] ,
[97, 5, 90, 60, 6, 13, 56, 82, 89, 79, 52, 34, 35, 100, 60, 34] ,
[15, 30, 53, 12, 79, 77, 15, 38, 100, 82, 35, 10, 24, 0, 29, 27] ,
[28, 66, 83, 1, 54, 89, 71, 35, 46, 62, 96, 26, 20, 58, 16, 81] ]

players_actions = [[0, 1], [0, 1], [0, 1],[0, 1]]

current_nfg=NFG(players_actions,utilities)
print("Utilities of the game")
print(current_nfg)

label = []
for i in range (len(current_nfg.players_actions)):
    label.append('Ax'.replace("x", str(i))) #'A _i'
for i in range (len(current_nfg.players_actions),2*(len(current_nfg.players_actions))):
    label.append('ax'.replace("x", str(i-len(current_nfg.players_actions)))) #'U _i'

tableau_final = np.concatenate((np.mat(label), np.concatenate( (current_nfg.joint_actions,current_nfg.disutilities.astype(int).astype(str)),axis=1)),axis=0)
print("Disutilities of the game:")
print(tableau_final)

## PCP: Polynomials creation

Using the disutilities, we compute the polynomials associated to each couple (player, action) to create the PCP associated to this game.

Here the variables yn_i will all be equals to 1.

In [ ]:
pcpglobal = PolynomialComplementaryProblem(current_nfg)
print(pcpglobal)
#MAnual print after removing the yn_g variables?

## Choosing an arbitrary joint strategy
Before using the path following algorthim an joint strategy must be chosen between among all the possible joint strategy. Meaning that for this example we have the choice between:
* (0,0,0,0)
* (0,0,0,1)
* (0,0,1,0)
* (0,0,1,1)
* (0,1,0,0)
* (0,1,0,1)
* (0,1,1,0)
* (0,1,1,1)
* (1,0,0,0)
* (1,0,0,1)
* (1,0,1,0)
* (1,0,1,1)
* (1,1,0,0)
* (1,1,0,1)
* (1,1,1,0)
* (1,1,1,1)

Here we will choose omega0 = (0, 0, 1, 1)

## Computing the first node:


In [ ]:

pcpglobal.omega0={0:0,1:0,2:1,3:1}
omega0=pcpglobal.omega0
xpairs_val = {}
for (n,i) in pcpglobal.couple_to_x.keys():
    if omega0[n]==i:
        xpairs_val[(n,i)] = 1
    else:
        xpairs_val[(n,i)] = 0

y_values={pcpglobal.ring("y0_0"):1, pcpglobal.ring("y1_0"):1, pcpglobal.ring("y2_0"):1}
print("The static value given to each variable xn_i for a couple (n,i) when current level< n are the following:")
print("xpairs_val:\n",xpairs_val)


# check first node computation:
node0 = first_node(pcpglobal,xpairs_val)[0]
#print("The corresponding first node is:\n",node0)
print("\nThe corresponding first node is:\n")
print(f"Level: {node0.level}")

var_pl=[]
var_notpl=[]
for (n,i) in node0.pcp.couple_to_x.keys():
    if n>node0.level:
        var_notpl+=[node0.pcp.couple_to_x[(n,i)]]
    else:
        var_pl+=[node0.pcp.couple_to_x[(n,i)]]
print(f"Variables of players at the current level: {var_pl}")
print(f"Variables of players above the current level:{var_notpl}")
print("Polynomials:")
for pol in node0.pcp.couple_to_poly.keys(): 
    print(" "+str(pol)+": "+str(node0.pcp.couple_to_poly[pol].subs(y_values)))
    
print(f"Set Z: {node0.set_z}")
print(f"Set W: {node0.set_w}")
print(f"Coordinate of the node: {node0.coordinates}")


## Complementary node at the level 0

For this node we have:
* Z=[(0,0)]
* W=[(0,1)]


Meaning that the node is complementary.

Given that the node is complementary at the level 0, we ascend the to level 1.


To reach the initial node at the level 1 for the node we found, we follow the arc (1,0) almost complementary where 
* Z=[(0,0),(1,1)]
* W=[(0,1)]

Because the initial joint strategy was (0,0,1,1) we must have the couple (1,1) in Z.

Considering that we are doing a lifting procedure we have two possible node we can reach, one for each possible couple (1,i) we can add to the set W:
* Z=[(0,0),(1,0)]
* W=[(0,1),(1,0)]

Or

* Z=[(0,0),(1,0)]
* W=[(0,1),(1,1)]

The node must be feasible, meaning that every value of the coordinate (variables xn_i) as well as the value of the polynomials An_i are >=0  


In [ ]:
degen_nodes=node0.lift(pcpglobal,xpairs_val)

print("First Node")
print(f"Set Z: {degen_nodes[0].set_z}")
print(f"Set W: {degen_nodes[0].set_w}")
print("Second Node")
print(f"Set Z: {degen_nodes[1].set_z}")
print(f"Set W: {degen_nodes[1].set_w}")

print(f"Coordinate of the nodes: {degen_nodes[0].coordinates}")
print(f"Coordinate of the nodes: {degen_nodes[1].coordinates}")

node1=degen_nodes[-1]

print("\nThe first degenarate feasible initial node is:\n")
present_node(node1,y_values)

## Degeneracy

The initial node is degenerate meaning that for both 
* Z=[(0,0),(1,1)]
* W=[(0,1),(1,0)]

and

* Z=[(0,0),(1,1)]
* W=[(0,1),(1,1)]

We have a possible node, both these nodes share the same coordinate.

We keep both of these node, chose to follow first the path of :

* Z=[(0,0),(1,1)]
* W=[(0,1),(1,1)]



## Arc traversal at the level 1

The almost complementary node is:
* Z=[(0,0),(1,1)]
* W=[(0,1),(1,1)]

The node found is almost complementary meaning that we must follow the arc that "leaves" from the node.

Given that the couple that "entered" a set was (1,1) in W, to try an reach a complementary node throught the (1,0) almost complementary path we remove (1,1) from the set Z meaning that we will follow the arc:

* Z=[(0,0)]
* W=[(0,1),(1,1)]

Which in turn means that the couple that can enter Z are :
* (1,0)

And enter W are:
* (0,0)
* (1,0)

There is 3 possible nodes:

* Z=[(0,0),(1,0)]
* W=[(0,1),(1,1)]

,

* Z=[(0,0)]
* W=[(0,1),(1,1),(0,0)]

and

* Z=[(0,0)]
* W=[(0,1),(1,1),(1,0)]

In [ ]:
degen_nodes_bis=node1.traverse(node0, xpairs_val)

print("First Node")
print(f"Set Z: {degen_nodes_bis[0].set_z}")
print(f"Set W: {degen_nodes_bis[0].set_w}")
print("Second Node")
print(f"Set Z: {degen_nodes_bis[1].set_z}")
print(f"Set W: {degen_nodes_bis[1].set_w}")

node2=degen_nodes_bis[-1]


#node2=degen_nodes[0].lift(pcpglobal,xpairs_val)[0]


print("\nThe complementary node is:\n")
present_node(node2,y_values)

## Degeneracy

The node is degenerate meaning that for both 
* Z=[(0,0)]
* W=[(0,1),(1,1),(0,0)]

and

* Z=[(0,0)]
* W=[(0,1),(1,1),(1,0)]

We have a possible node, both these nodes share the same coordinate.

We keep both of these node, chose to follow first the path of :

* Z=[(0,0)]
* W=[(0,1),(1,1),(1,0)]

## Lifing procedure from the level 1 to 2

Considering the joint strategy chosen initialy we will follow the arc:
* Z=[(0,0),(2,0)]
* W=[(0,1),(1,1),(1,0)]

Wich means that we will consider the following nodes:
* Z=[(0,0),(2,0)]
* W=[(0,1),(1,1),(1,0),(2,0)]

And

* Z=[(0,0),(2,0)]
* W=[(0,1),(1,1),(1,0),(2,1)]


In [ ]:

node3=node2.lift(pcpglobal, xpairs_val)
print("No reachable node")
print(node3)
print("BACKTRACK")
#present_node(node3,y_values)

## Degeneracy: BACKTRACK

There is no reachable node after the lift procedure meaning that we backtrack to the previously encountered degenerate node which is:

* Z=[(0,0)]
* W=[(0,1),(1,1),(0,0)]


## Arc traversal at the level 1

The almost complementary node is:
* Z=[(0,0)]
* W=[(0,1),(1,1),(0,0)]

The node found is almost complementary meaning that we must follow the arc that "leaves" from the node.

Given that the couple that "entered" a set was (0,0) in W, to try an reach a complementary node throught the (1,0) almost complementary path we remove (0,0) from the set Z meaning that we will follow the arc:

* Z=[]
* W=[(0,1),(1,1),(0,0)]

Which in turn means that the couple that can enter Z are :
* (0,1)
* (1,0)
* (1,1)

and in W:
* (1,0)

There is 4 possible nodes:

* Z=[(0,1)]
* W=[(0,1),(1,1),(0,0)]

,

* Z=[(1,0)]
* W=[(0,1),(1,1),(0,0)]

,

* Z=[(1,1)]
* W=[(0,1),(1,1),(0,0)]

and

* Z=[]
* W=[(0,1),(1,1),(0,0),(1,0)]



In [ ]:
node2bis=degen_nodes_bis[-2]
node3=node2bis.traverse(node1, xpairs_val)[0]


present_node(node3,y_values)


## Arc traversal at the level 1

The almost complementary node is:

* Z=[(0,1)]
* W=[(0,1),(1,1),(0,0)]

The node found is almost complementary meaning that we must follow the arc that "leaves" from the node.

Given that the couple that "entered" a set was (0,1) in Z, to try an reach a complementary node throught the (1,0) almost complementary path we remove (0,1) from the set W meaning that we will follow the arc:
* Z=[(0,1)]
* W=[(1,1),(0,0)]

Which means that the couple that can enter Z are :
* (1,0)
* (1,1)

and in W:
* (1,0)

There is 3 possible node

* Z=[(0,1),(1,0)]
* W=[(1,1),(0,0)]

,

* Z=[(0,1),(1,1)]
* W=[(1,1),(0,0)]

and

* Z=[(0,1)]
* W=[(1,1),(0,0),(1,0)]



In [ ]:
node4=node3.traverse(node2bis, xpairs_val)[0]


present_node(node4,y_values)

## Lift from level 1 to level 2

Considering the joint strategy chosen initialy we will follow the arc:
* Z=[(0,1),(1,0),(2,0)]
* W=[(1,1),(0,0)]

Wich means that we will consider the following nodes:
* Z=[(0,1),(1,0),(2,0)]
* W=[(1,1),(0,0),(2,0)]

And

* Z=[(0,1),(1,0),(2,0)]
* W=[(1,1),(0,0),(2,1)]

In [ ]:
node5=node4.lift(pcpglobal,xpairs_val)[0]


present_node(node5,y_values)


## Arc traversal at the level 2

The almost complementary node is:

* Z=[(0,1),(1,0),(2,0)]
* W=[(1,1),(0,0),(2,0)]

The node found is almost complementary meaning that we must follow the arc that "leaves" from the node.

Given that the couple that "entered" a set was (2,0) in W, to try an reach a complementary node throught the (2,1) almost complementary path we remove (2,0) from the set Z meaning that we will follow the arc:
* Z=[(0,1),(1,0)]
* W=[(1,1),(0,0),(2,0)]

Which means that the couple that can enter Z are :
* (2,1)

and in W:
* (0,1)
* (1,0)
* (2,1)

There is 4 possible nodes:

* Z=[(0,1),(1,0),(2,1)]
* W=[(1,1),(0,0),(2,0)]

,

* Z=[(0,1),(1,0)]
* W=[(1,1),(0,0),(2,0),(0,1)]

,

* Z=[(0,1),(1,0)]
* W=[(1,1),(0,0),(2,0),(1,0)]

and

* Z=[(0,1),(1,0)]
* W=[(1,1),(0,0),(2,0),(2,1)]



In [ ]:
node6=node5.traverse(node4, xpairs_val)[0]

present_node(node6,y_values)

## Arc traversal at the level 2

The almost complementary node is:

* Z=[(0,1),(1,0)]
* W=[(1,1),(0,0),(2,0),(0,1)]

The node found is almost complementary meaning that we must follow the arc that "leaves" from the node.

Given that the couple that "entered" a set was (0,1) in W, to try an reach a complementary node throught the (2,1) almost complementary path we remove (0,1) from the set Z meaning that we will follow the arc:
* Z=[(1,0)]
* W=[(1,1),(0,0),(2,0),(0,1)]

Which means that the couple that can enter Z are :
* (0,0)
* (2,0)
* (2,1)

and in W:
* (1,0)
* (2,1)

There is 5 possible node:

* Z=[(1,0),(0,0)]
* W=[(1,1),(0,0),(2,0),(0,1)]

,

* Z=[(1,0),(2,0)]
* W=[(1,1),(0,0),(2,0),(0,1)]

,

* Z=[(1,0),(2,1)]
* W=[(1,1),(0,0),(2,0),(0,1)]

,

* Z=[(1,0)]
* W=[(1,1),(0,0),(2,0),(0,1),(1,0)]

and

* Z=[(1,0)]
* W=[(1,1),(0,0),(2,0),(0,1),(2,1)]


In [ ]:
node7=node6.traverse(node5, xpairs_val)[0]

present_node(node7,y_values)

## Lift from level 2 to level 3

Considering the joint strategy chosen initialy we will follow the arc:
* Z=[(1,0),(3,0)]
* W=[(1,1),(0,0),(2,0),(0,1),(2,1)]

Wich means that we will consider the following nodes:
* Z=[(1,0),(3,0)]
* W=[(1,1),(0,0),(2,0),(0,1),(2,1),(3,0)]

And

* Z=[(1,0),(3,0)]
* W=[(1,1),(0,0),(2,0),(0,1),(2,1),(3,1)]

In [ ]:
node8=node7.lift(pcpglobal,xpairs_val)[0]


present_node(node8,y_values)


## Complementary node found

* Z=[(1,0),(3,0)]
* W=[(1,1),(0,0),(2,0),(0,1),(2,1),(3,1)]

The node found is complementary and we are at the last level, meaning that this node is a solution to the PCP which correspond to mixed nash equilibrium of the corresponding game.

The coordinate of the nodes are:

In [ ]:
for (n,i) in pcpglobal.couple_to_x.keys():
    print(f"{pcpglobal.couple_to_x[(n,i)]} = {node8.coordinates[pcpglobal.couple_to_x[(n,i)]]} = {node8.coordinates[pcpglobal.couple_to_x[(n,i)]].radical_expression()}")

In [ ]:
mixed_strat=node8.normalized_strategy()
for n in mixed_strat.keys():
    print(f"Player {n}: {mixed_strat[n]}")

# Other possible path that reach a solution (partial description and execution )

Let's consider the other possible degenerate node first encountered

## Lift from level 1 to level 2

Considering the joint strategy chosen initialy we will follow the arc:
* Z=[(0,0),(1,1),(2,0)]
* W=[(0,1),(1,0)]

Wich means that we will consider the following nodes:
* Z=[(0,0),(1,1),(2,0)]
* W=[(0,1),(1,0),(2,0)]

And

* Z=[(0,0),(1,1),(2,0)]
* W=[(0,1),(1,0),(2,1)]

In [ ]:
node1_bis=degen_nodes[0]

alt_degen=node1_bis.lift(pcpglobal, xpairs_val)
print("First Node")
print(f"Set Z: {alt_degen[0].set_z}")
print(f"Set W: {alt_degen[0].set_w}")
print("Second Node")
print(f"Set Z: {alt_degen[1].set_z}")
print(f"Set W: {alt_degen[1].set_w}")


alt_node2=alt_degen[0]


print("\nThe almost complementary node is:\n")
present_node(alt_node2,y_values)

## Degeneracy

There is 2 reachable node with the same coordinate

* Z=[(0,0),(1,1),(2,0)]
* W=[(0,1),(1,0),(2,0)]

and

* Z=[(0,0),(1,1),(2,0)]
* W=[(0,1),(1,0),(1,1)]

Let's choose:

* Z=[(0,0),(1,1),(2,0)]
* W=[(0,1),(1,0),(2,0)]

## Traversal at the level 2 

Arc followed:
* Z=[(0,0),(1,1)]
* W=[(0,1),(1,0),(2,0)]

Possible nodes:

* Z=[(0,0),(1,1),(2,1)]
* W=[(0,1),(1,0),(2,0)]

,

* Z=[(0,0),(1,1)]
* W=[(0,1),(1,0),(2,0),(0,0)]

,

* Z=[(0,0),(1,1)]
* W=[(0,1),(1,0),(2,0),(1,1)]

and 

* Z=[(0,0),(1,1)]
* W=[(0,1),(1,0),(2,0),(2,1)]


In [ ]:

alt_degen2=alt_node2.traverse(node1_bis, xpairs_val)
print("First Node")
print(f"Set Z: {alt_degen2[0].set_z}")
print(f"Set W: {alt_degen2[0].set_w}")
print("Second Node")
print(f"Set Z: {alt_degen2[1].set_z}")
print(f"Set W: {alt_degen2[1].set_w}")

alt_node3=alt_degen2[0]

present_node(alt_node3,y_values)

## Degeneracy

There is 2 reachable node with the same coordinate

* Z=[(0,0),(1,1)]
* W=[(0,1),(1,0),(2,0),(1,1)]

and

* Z=[(0,0),(1,1),(2,0)]
* W=[(0,1),(1,0),(2,0)]

Lets choose :

* Z=[(0,0),(1,1)]
* W=[(0,1),(1,0),(2,0),(1,1)]

## Traversal at the level 2

Arc:

* Z=[(0,0)]
* W=[(0,1),(1,0),(2,0),(1,1)]

Possible arc:

* Z=[(0,0),(1,0)]
* W=[(0,1),(1,0),(2,0),(1,1)]

,

* Z=[(0,0),(2,0)]
* W=[(0,1),(1,0),(2,0),(1,1)]


,

* Z=[(0,0),(2,1)]
* W=[(0,1),(1,0),(2,0),(1,1)]

,

* Z=[(0,0)]
* W=[(0,1),(1,0),(2,0),(1,1),(0,0)]

,

* Z=[(0,0)]
* W=[(0,1),(1,0),(2,0),(1,1),(2,1)]







In [ ]:


alt_node4=alt_node3.traverse(alt_node2, xpairs_val)[0]


print("\nThe complementary node is:\n")
present_node(alt_node4,y_values)

## Traversal at the level 2

Arc:

* Z=[]
* W=[(0,1),(1,0),(2,0),(1,1),(0,0)]

Possible node

* Z=[(0,1)]
* W=[(0,1),(1,0),(2,0),(1,1),(0,0)]

* Z=[(1,0)]
* W=[(0,1),(1,0),(2,0),(1,1),(0,0)]

* Z=[(1,1)]
* W=[(0,1),(1,0),(2,0),(1,1),(0,0)]

* Z=[(2,0)]
* W=[(0,1),(1,0),(2,0),(1,1),(0,0)]

* Z=[(2,1)]
* W=[(0,1),(1,0),(2,0),(1,1),(0,0)]

* Z=[]
* W=[(0,1),(1,0),(2,0),(1,1),(0,0),(2,1)]




In [ ]:
alt_node5=alt_node4.traverse(alt_node3, xpairs_val)[0]


print("\nThe complementary node is:\n")
present_node(alt_node5,y_values)

## Descent from the level 2 to level 1

* Z=[]
* W=[(0,1),(1,0),(1,1),(0,0)]

In [ ]:
alt_node6=alt_node5.descend(pcpglobal, xpairs_val)[0]


print("\nThe complementary node is:\n")
present_node(alt_node6,y_values)

## Traversal at the level 1

Arc:
* Z=[]
* W=[(0,1),(1,1),(0,0)]

Possible node:
* Z=[(0,0)]
* W=[(0,1),(1,1),(0,0)]

* Z=[(0,1)]
* W=[(0,1),(1,1),(0,0)]

* Z=[(1,0)]
* W=[(0,1),(1,1),(0,0)]

* Z=[(1,1)]
* W=[(0,1),(1,1),(0,0)]



In [ ]:
alt_node7=alt_node6.traverse(alt_node5, xpairs_val)[0]


print("\nThe almost complementary node is:\n")
present_node(alt_node7,y_values)

## Traversal at the level 1

Arc

* Z=[(0,1)]
* W=[(1,1),(0,0)]

Possible node:

* Z=[(0,1),(1,0)]
* W=[(1,1),(0,0)]

* Z=[(0,1),(1,1)]
* W=[(1,1),(0,0)]

* Z=[(0,1)]
* W=[(1,1),(0,0),(1,0)]



In [ ]:
alt_node8=alt_node7.traverse(alt_node6, xpairs_val)[0]


print("\nThe almost complementary node is:\n")
present_node(alt_node8,y_values)

## Lift from level 1 to level 2

Considering the joint strategy chosen initialy we will follow the arc:
* Z=[(0,1),(1,0),(2,0)]
* W=[(1,1),(0,0)]

Wich means that we will consider the following nodes:
* Z=[(0,1),(1,0),(2,0)]
* W=[(1,1),(0,0),(2,0)]

And

* Z=[(0,1),(1,0),(2,0)]
* W=[(1,1),(0,0),(2,1)]

Which lead to the same path as the other degenerate node

In [ ]:
alt_node9=alt_node8.lift(pcpglobal, xpairs_val)[0]

print("\nThe almost complementary initial node is:\n")
present_node(alt_node9,y_values)